In [19]:
import pandas as pd
import pickle

# Load matrix

In [20]:
#movie_cosine_matrix = pickle.load(open("./books_cosines_matrix.sav", "rb"))

# Calculate

In [21]:
def gd_path(file_id):
    return f"https://drive.google.com/uc?export=download&id={file_id}"


files_id = {
    'rating': "1F4_-HBPBSySMjxdGxlykWVjvVn9AJ0BS",
    'movies': "1PDuCaAhhVTRLYdftMr6VqX23crMqB_qg",
    }

rating = pd.read_csv(gd_path(files_id['rating']), sep=",")
movies = pd.read_csv(gd_path(files_id['movies']), sep=",")


In [22]:
user_movie_matrix = pd.pivot_table(data=rating,
                                  values='rating',
                                  index='userId',
                                  columns='movieId',
                                  fill_value=0)

In [23]:
from sklearn.metrics.pairwise import cosine_similarity


movies_cosines_matrix = pd.DataFrame(cosine_similarity(user_movie_matrix.T),
                                    columns=user_movie_matrix.columns,
                                    index=user_movie_matrix.columns)

In [24]:
lovely_bones_title_mask = movies["title"].str.contains('Toy Story', case=False)
lovely_bones_author_mask = movies["genres"].str.contains('Comedy', case=False)
lovely_bones_isbn = movies.loc[lovely_bones_title_mask & lovely_bones_author_mask, "movieId"].values[0]

# Select the column, from the above matrix, matching the movieId of Toy Story
lovely_bones_correlations_df = pd.DataFrame(movies_cosines_matrix[lovely_bones_isbn])

In [26]:
lovely_bones_cosines_df = pd.DataFrame(movies_cosines_matrix[lovely_bones_isbn])

# Rename the column 'lovely_bones_isbn' to 'lovely_bones_cosine'
lovely_bones_cosines_df = lovely_bones_cosines_df.rename(columns={lovely_bones_isbn: 'lovely_bones_cosine'})

# Remove the row with the index 'lovely_bones_isbn'
lovely_bones_cosines_df = lovely_bones_cosines_df[lovely_bones_cosines_df.index != lovely_bones_isbn]

# Sort the 'lovely_bones_cosines_df' by the column 'lovely_bones_cosine' column in descending order.
lovely_bones_cosines_df = lovely_bones_cosines_df.sort_values(by="lovely_bones_cosine", ascending=False)

# Find out the number of users rated both The Lovely Bones and the other book
no_of_users_rated_both_books = [sum((user_movie_matrix[lovely_bones_isbn] > 0) & (user_movie_matrix[isbn] > 0)) for isbn in lovely_bones_cosines_df.index]

# Create a column for the number of users who rated The Lovely Bones and the other book
lovely_bones_cosines_df['users_who_rated_both_movies'] = no_of_users_rated_both_books

# Remove recommendations that have less than 10 users who rated both books.
lovely_bones_cosines_df = lovely_bones_cosines_df[lovely_bones_cosines_df["users_who_rated_both_movies"] > 10]

# Display the first 10 rows of the 'lovely_bones_cosines_df' DataFrame.
lovely_bones_cosines_df.head(10)

,lovely_bones_cosine,users_who_rated_both_movies
movieId,,
3114,0.572601,81
480,0.565637,132
780,0.564262,122
260,0.557388,134
356,0.547096,154
364,0.541145,107
1210,0.541089,116
648,0.538913,106
1265,0.534169,97
